In [ ]:
#반복문 버전 ( 단일변수 활용)
#### 2) 모멘텀이 있는 역전파 학습 수행하기


#함수 선택부

set_lr_decay_func(fixed_decay)  #학습률 조정 함수 선택
set_activ_func(logi_func) #활성화 함수 선택
set_grad_func(logi_func_deriv)  #활성화 함수 도함수 선택
set_cost_func(sse_cost) #비용함수 선택
set_weight_init_func(xavier_initialization) #가중치 초기화 함수 선택

# 가중치 초기화
weights = []  #가중치 저장용 리스트
for(i) in range(len(layer_sizes)-1):
    weights.append(call_weight_init_func(layer_sizes[i]+1, layer_sizes[i+1]))
# 학습률 및 모멘텀 변수 초기화

initial_lr = 0.004 # 초기 학습률
decay_rate = 0.0001 # 학습률 감소율
momentum = 0.84 # 모멘텀

# 모멘텀을 위한 가중치 업데이트 변수 초기화
v_w = []
for(i) in range(len(weights)):
    v_w.append(np.zeros_like(weights[i]))
# 배치 크기 설정
batch_size = 64  # 또는 당신의 데이터셋에 적합한 다른 크기
num_batches = int(np.ceil(X.shape[0] / batch_size))

# 종료 조건 설정
n_epochs =1000 
gradient_threshold = 1e-8
target_cost = 650

J_history = [J]
print("weights are initialized by " + cur_weight_init_func.__name__)
for epoch in range(n_epochs):

    # 학습률 동적 조정
    alpha = call_lr_decay_func(initial_lr, epoch, decay_rate, n_epochs)

    # 데이터를 무작위로 섞기
    indices = np.arange(X.shape[0])  # X의 길이만큼 인덱스 배열 생성
    np.random.shuffle(indices)       # 인덱스 배열을 무작위로 섞음
    X_shuffled = X_arr[indices]          # 섞인 인덱스에 따라 X를 섞음
    y_shuffled = y_arr[indices]          # 섞인 인덱스에 따라 y를 섞음

    for batch in range(num_batches):
        # 배치 데이터 가져오기
        start = batch * batch_size
        end = min(start + batch_size, X_arr.shape[0])
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]

        # 편향 항 추가
        y_onehot_batch = one_hot_encode(y_batch, 10)

        # 비용의 순전파
        a3, a2, a1, a0, in3, in2, in1 = front_propag(weights[0], weights[1], weights[2], X_batch, y_batch)

        # 오차의 역전파
        back_propag(a3, a2, a1, a0, in3, in2, in1, X_batch, y_onehot_batch)

        grad23, grad12, grad01 = back_propag(a3, a2, a1, a0,in3, in2, in1, X_batch, y_onehot_batch)
        grads = [grad01, grad12, grad23]

        # # # 모멘텀을 계산
        for i in range(len(weights)):
            v_w[i] = momentum * v_w[i] + alpha * grads[i]
        
        # 가중치 업데이트
        for i in range(len(weights)):
            weights[i] -= v_w[i]

    if epoch % 100 == 0:
      # 에포크의 끝에서 손실 계산 및 출력
      #J = total_cost_func(w01, w12,w23, X, y)
      J = total_cost_func(weights[0], weights[1],weights[2], X, y)
      print(f"Epoch {epoch}: Cost = {J} LR = {alpha}")
        
      # 그래디언트 크기가 임계값 이하면 학습 종료
      if np.linalg.norm(grad01) + np.linalg.norm(grad12) + np.linalg.norm(grad23) < gradient_threshold:
          print(f"Stopped at epoch {epoch}: Gradient norm below threshold.")
          break
      # 오차가 target_cost 미만이면 학습 종료 (거의 최적)
    
      if J < target_cost:
          print(f"Stopped at epoch {epoch}: Cost below {target_cost}.")
          break
    #J_history.append(J)

print(f"Epoch {epoch + 1}: Cost = {J} ")


In [ ]:
## 비 반복문 버전 

#### 2) 모멘텀이 있는 역전파 학습 수행하기

# # 랜덤한 실수로 가중치를 초기화
# w01 = np.random.rand(w01_size[0], w01_size[1])
# w12 = np.random.rand(w12_size[0], w12_size[1])
# w23 = np.random.rand(w23_size[0], w23_size[1])
# print("w01 and w12 are all initialized with random values in [0, 1].")

#불러오기

#w01, w12,w23 = load_weights("my_model_weights.npz")

#함수 선택부

set_lr_decay_func(fixed_decay)  #학습률 조정 함수 선택
set_activ_func(logi_func) #활성화 함수 선택
set_grad_func(logi_func_deriv)  #활성화 함수 도함수 선택
set_cost_func(sse_cost) #비용함수 선택
set_weight_init_func(xavier_initialization) #가중치 초기화 함수 선택

# 가중치 초기화
print (w01_size, w12_size, w23_size)
w01 = call_weight_init_func(w01_size[0], w01_size[1])
w12 = call_weight_init_func(w12_size[0], w12_size[1])
w23 = call_weight_init_func(w23_size[0], w23_size[1])

# 학습률 및 모멘텀 변수 초기화

initial_lr = 0.004 # 초기 학습률
decay_rate = 0.0001 # 학습률 감소율
momentum = 0.84 # 모멘텀

print("vectorized implementation")
v_w01 = np.zeros_like(w01)
v_w12 = np.zeros_like(w12)
v_w23 = np.zeros_like(w23)

# 배치 크기 설정
batch_size = 64  # 또는 당신의 데이터셋에 적합한 다른 크기
num_batches = int(np.ceil(X.shape[0] / batch_size))

# 종료 조건 설정
n_epochs = 5000
gradient_threshold = 1e-8
target_cost = 650

J_history = [J]
for epoch in range(n_epochs):

    # 학습률 동적 조정
    alpha = call_lr_decay_func(initial_lr, epoch, decay_rate, n_epochs)

    # 데이터를 무작위로 섞기
    indices = np.arange(X.shape[0])  # X의 길이만큼 인덱스 배열 생성
    np.random.shuffle(indices)       # 인덱스 배열을 무작위로 섞음
    X_shuffled = X_arr[indices]          # 섞인 인덱스에 따라 X를 섞음
    y_shuffled = y_arr[indices]          # 섞인 인덱스에 따라 y를 섞음

    for batch in range(num_batches):
        # 배치 데이터 가져오기
        start = batch * batch_size
        end = min(start + batch_size, X_arr.shape[0])
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]

        # 편향 항 추가
        y_onehot_batch = one_hot_encode(y_batch, 10)

        # 비용의 순전파
        a3, a2, a1, a0, in3, in2, in1 = front_propag(w01, w12, w23, X_batch, y_batch)

        # 오차의 역전파
        grad23, grad12, grad01 = back_propag(a3, a2, a1, a0,in3, in2, in1, X_batch, y_onehot_batch)

        # 모멘텀을 계산
        v_w23 = momentum * v_w23 + alpha * grad23
        v_w12 = momentum * v_w12 + alpha * grad12
        v_w01 = momentum * v_w01 + alpha * grad01

        # 가중치 업데이트
        w23 -= v_w23
        w12 -= v_w12
        w01 -= v_w01

    if epoch % 100 == 0:
      # 에포크의 끝에서 손실 계산 및 출력
      J = total_cost_func(w01, w12,w23, X, y)
      print(f"Epoch {epoch}: Cost = {J} LR = {alpha}")

        
      # 그래디언트 크기가 임계값 이하면 학습 종료
      if np.linalg.norm(grad01) + np.linalg.norm(grad12) + np.linalg.norm(grad23) < gradient_threshold:
          print(f"Stopped at epoch {epoch}: Gradient norm below threshold.")
          break

      # 오차가 target_cost 미만이면 학습 종료 (거의 최적)
    
      if J < target_cost:
          print(f"Stopped at epoch {epoch}: Cost below {target_cost}.")
          break
      # J_history.append(J)

print(f"Epoch {epoch + 1}: Cost = {J} ")

